<a href="https://colab.research.google.com/github/bryanbayup/Machine-Learning/blob/main/Petpoint2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install Sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 6.6 MB/s eta 0:00:00


In [3]:
import pandas as pd
import re
import string
# Mengimpor Sastrawi
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Membaca dataset
df = pd.read_csv('dog_cat.csv')

# Fungsi untuk membersihkan teks
def clean_text(text):
    text = text.lower()  # Mengubah teks menjadi huruf kecil
    text = re.sub(r'\d+', '', text)  # Menghapus angka
    text = text.translate(str.maketrans('', '', string.punctuation))  # Menghapus tanda baca
    text = text.strip()  # Menghapus spasi berlebih
    return text

# Fungsi untuk menghapus stopwords menggunakan Sastrawi
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopword_remover = factory.create_stop_word_remover()
    cleaned_text = stopword_remover.remove(text)
    return cleaned_text

# Menerapkan fungsi ke kolom Gejala dan Penanganan Pertama
df['Cleaned_Gejala'] = df['Gejala'].apply(clean_text).apply(remove_stopwords)
df['Cleaned_Penanganan'] = df['Penanganan Pertama'].apply(clean_text).apply(remove_stopwords)

# Menampilkan hasil preprocessing
print(df[['Gejala', 'Cleaned_Gejala']].head())

                                  Gejala                      Cleaned_Gejala
0        Demam, muntah, diare, dehidrasi        demam muntah diare dehidrasi
1      Perubahan perilaku, agresi, demam     perubahan perilaku agresi demam
2  Demam, muntah, gagal ginjal, kelesuan  demam muntah gagal ginjal kelesuan
3          Muntah, diare berdarah, lemas         muntah diare berdarah lemas
4     Batuk, pilek, demam tinggi, kejang     batuk pilek demam tinggi kejang


In [4]:
from sklearn.preprocessing import LabelEncoder

# Mengubah nama penyakit menjadi label numerik
le_disease = LabelEncoder()
df['Label_Penyakit'] = le_disease.fit_transform(df['Nama Penyakit'])

# Mengubah nama hewan menjadi label numerik
le_animal = LabelEncoder()
df['Label_Hewan'] = le_animal.fit_transform(df['Nama Hewan'])

In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Menggabungkan semua teks gejala
all_text = df['Cleaned_Gejala'].tolist()

# Membuat TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(all_text)

# Label penyakit sebagai target
y = df['Label_Penyakit']

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

In [7]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Konversi sparse matrix ke dense matrix
X_train_dense = X_train.toarray()
X_test_dense = X_test.toarray()

# Mendapatkan jumlah fitur
input_dim = X_train_dense.shape[1]

# Membangun model
model = Sequential()
model.add(Dense(128, input_shape=(input_dim,), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(len(le_disease.classes_), activation='softmax'))

# Mengkompilasi model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Menampilkan ringkasan model
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           4,224 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 10)                  │             650 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 13,130 (51.29 KB)

 Trainable params: 13,130 (51.29 KB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# Melatih model
epochs = 50
batch_size = 64

history = model.fit(X_train_dense, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test_dense, y_test))

Epoch 1/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 3s 41ms/step - accuracy: 0.1421 - loss: 2.2973 - val_accuracy: 0.6500 - val_loss: 2.1967
Epoch 2/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.4109 - loss: 2.1924 - val_accuracy: 1.0000 - val_loss: 2.0798
Epoch 3/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6729 - loss: 2.0669 - val_accuracy: 1.0000 - val_loss: 1.9402
Epoch 4/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8638 - loss: 1.9202 - val_accuracy: 1.0000 - val_loss: 1.7687
Epoch 5/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9300 - loss: 1.7786 - val_accuracy: 1.0000 - val_loss: 1.5641
Epoch 6/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9583 - loss: 1.6027 - val_accuracy: 1.0000 - val_loss: 1.3284
Epoch 7/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9695 - loss: 1.3552 - val_accuracy: 1.0000 - val_loss: 1.0678
Epoch 8/50
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9856 - loss: 1.1360 - val_accuracy: 1.0000 - val_loss: 0.8077
Epoch 9

In [9]:
# Evaluasi model
loss, accuracy = model.evaluate(X_test_dense, y_test)
print(f'Akurasi model: {accuracy * 100:.2f}%')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 1.0000 - loss: 6.3595e-04 
Akurasi model: 100.00%


In [10]:
def detect_animal_type(user_input):
    animal_types = ['kucing', 'anjing']
    for animal in animal_types:
        if animal in user_input.lower():
            return animal
    return None

In [11]:
def ask_for_animal_type():
    while True:
        animal_type = input("Chatbot: Hewan apa yang Anda miliki? (kucing/anjing): ").lower()
        if animal_type in ['kucing', 'anjing']:
            return animal_type
        else:
            print("Chatbot: Maaf, saya hanya bisa memberikan saran untuk kucing atau anjing.")
            return None

In [12]:
def predict_disease(symptom_text, animal_type):
    if animal_type not in ['kucing', 'anjing']:
        return None, None
    # Filter dataset berdasarkan jenis hewan
    df_animal = df[df['Nama Hewan'].str.lower() == animal_type]
    if df_animal.empty:
        return None, None
    # Preprocessing input pengguna
    processed_text = clean_text(symptom_text)
    processed_text = remove_stopwords(processed_text)
    vectorized_text = vectorizer.transform([processed_text]).toarray()
    # Prediksi penyakit
    prediction = model.predict(vectorized_text)
    predicted_label = prediction.argmax(axis=-1)[0]
    disease_name = le_disease.inverse_transform([predicted_label])[0]
    # Pastikan penyakit tersebut ada dalam data hewan yang dipilih
    if disease_name not in df_animal['Nama Penyakit'].values:
        return None, None
    return disease_name, df_animal

In [13]:
def get_first_aid(disease_name, df_animal):
    penanganan = df_animal[df_animal['Nama Penyakit'] == disease_name]['Penanganan Pertama'].iloc[0]
    return penanganan

In [14]:
def chatbot_response(user_input):
    # Mendeteksi jenis hewan
    animal_type = detect_animal_type(user_input)
    if not animal_type:
        # Jika tidak ditemukan, minta pengguna untuk menyebutkan jenis hewan
        animal_type = ask_for_animal_type()
        if not animal_type:
            return "Maaf, saya hanya bisa memberikan saran untuk kucing atau anjing."
    # Prediksi penyakit
    disease_name, df_animal = predict_disease(user_input, animal_type)
    if not disease_name:
        return f"Maaf, saya tidak dapat menemukan penyakit yang sesuai untuk gejala tersebut pada {animal_type}."
    # Mendapatkan penanganan pertama
    first_aid = get_first_aid(disease_name, df_animal)
    response = f"Sepertinya {animal_type} Anda sakit **{disease_name}**. {first_aid}"
    return response

In [15]:
while True:
    user_input = input("Anda: ")
    if user_input.lower() in ['exit', 'quit', 'keluar']:
        print("Chatbot: Terima kasih, semoga membantu!")
        break
    response = chatbot_response(user_input)
    print("Chatbot:", response)

Anda: kucing saya mengalami muntah
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Chatbot: Sepertinya kucing Anda sakit **Panleukopenia**. Isolasi kucing yang sakit, beri air minum yang banyak, jaga kehangatan tubuhnya, dan segera ke dokter hewan jika kondisinya tidak membaik.
Anda: anjing saya mengalami gejala mengeluarkan banyak liur
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
Chatbot: Maaf, saya tidak dapat menemukan penyakit yang sesuai untuk gejala tersebut pada anjing.
Anda: anjing saya mengalami gejala Batuk, pilek, demam tinggi, kejang
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Chatbot: Sepertinya anjing Anda sakit **Distemper**. Isolasi anjing, berikan kompres hangat untuk mengurangi demam, berikan cairan, dan pantau gejala pernapasan, segera bawa ke dokter hewan.


KeyboardInterrupt: Interrupted by user

In [16]:
# Setelah melatih model
model.save('chatbot_model.h5')